In [ ]:
import arcpy
from arcpy import *
from arcpy.sa import *
from arcpy.ia import *
import os

areas = ["DENTONHILLS", 'VICTORIA', 'WRIGHT', 'TAYLOR']

arcpy.env.workspace       = 'D:\\PROJECT_ANTARCTICA\\antarctica\\antarctica_arcgis\\antarctica_arcgis.gdb\\'
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

for area in areas:
    folder = 'D:\\PROJECT_ANTARCTICA\\antarctica\\' + area + "\\"
    in_point = folder + area + "_points.shp"
    dem = Raster('tandemx_dem')
    cti = Raster('tandemx_cti')
    slope = Raster('tandemx_slope')
    aspect = Raster('tandemx_aspect')
    rel_aspect = Con(aspect, aspect, (360-aspect), "VALUE <= 180")
    rr100 = Raster('rel_relief_100m')
    rr1000 = Raster('rel_relief_1000m')
    
    ## extract multi values into points
    ExtractMultiValuesToPoints(in_point, [[dem, "ELEVATION"], [cti, "CTI"], [slope, "SLOPE"], [aspect, "ASPECT"], [rel_aspect, "REL_ASPECT"], [rr100, "RR_100"], [rr1000, "RR_1000"]])
    
    bio_attributes = ['CYANO', 'MOSS', 'LICHEN', 'SPRINGTAILS', 'MITES']
    ## FBCR train and predict
    for bio in bio_attributes:
        prediction_type = "PREDICT_FEATURES"
        in_features = "NZTAB_SURVEYPOINTS"
        variable_predict = "COV_"+bio
        treat_variable_as_categorical = "NUMERIC" 
        explanatory_variables = [["N_DEM", "false"], ["N_CTI", "false"], ["N_SLOPE", "false"], ["N_ASPECT", "false"], ["N_RASPECT", "false"], ["RR_100", "false"], ["RR_1000", "false"]]
        distance_features = None
        explanatory_rasters = None
        features_to_predict = in_point
        output_features = folder + area + "_FBCR_PREDICTED_"+ bio+".shp"
        output_raster = None
        explanatory_variable_matching = [["ELEVATION", "false"], ["CTI", "false"], ["SLOPE", "false"], ["ASPECT", "false"], ["REL_ASPECT", "false"], ["RR_100", "false"], ["RR_1000", "false"]]
        explanatory_distance_matching = None
        explanatory_rasters_matching = None
        output_trained_features = None
        output_importance_table = folder + area + "_FBCR_VI_"+ bio+".shp"
        use_raster_values = True
        number_of_trees = 300
        minimum_leaf_size = 30
        maximum_level = None
        sample_size = 100
        random_sample = None
        percentage_for_training = 10
        
        arcpy.stats.Forest(prediction_type, in_features, variable_predict,
                            treat_variable_as_categorical, explanatory_variables, distance_features,
                            explanatory_rasters, features_to_predict, output_features, output_raster,
                            explanatory_variable_matching, explanatory_distance_matching, 
                            explanatory_rasters_matching, output_trained_features, output_importance_table,
                            use_raster_values, number_of_trees, minimum_leaf_size, maximum_level,
                            sample_size, random_sample, percentage_for_training)
